In [1]:
#Only good for static websites
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#https://ece.ucdavis.edu/undergraduate/undergraduate-advising

url = "https://ece.ucdavis.edu/undergraduate/undergraduate-advising"
# base_url = "https://catalog.ucdavis.edu"

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [3]:
#All the links and names associated with the bubble icons on the site

links = soup.find_all('div', class_ = 'layout__region priority-links')
titles_scraped = soup.find_all('div', class_ = 'vertical-link__title')
titles = []

for title in titles_scraped:
    stripped = str(title).split('>')[1]
    if 'New Students' not in str(stripped).replace('&amp;', "").replace('</div', ''):
        if str(stripped).replace('&amp;', "").replace('</div', '').replace('\n', '').replace('  ', '') == 'DegreesRequirements':
            titles.append("Degree Requirements")
        else:
            titles.append(str(stripped).replace('&amp;', "").replace('</div', '').replace('\n', '').replace('  ', ''))


list_of_scraped_links = {'Advising for New Students': 'https://ece.ucdavis.edu/undergraduate/aggie-advising',
                         'IDP Program': 'https://ece.ucdavis.edu/undergraduate/integrated-degree-programs',
                         'College of Engineering': 'https://engineering.ucdavis.edu/undergraduates/academic-advising'}

i = 0
for scrape in links:
    for a in scrape.find_all('a', href=True):
        if "https" in a['href']:
            # print("Found the URL:", a['href'])
            list_of_scraped_links[titles[i]] = a['href']
            i = i + 1
    
    # print(len(list_of_scraped_links))


list_of_scraped_links['Change of Major'] = 'https://ece.ucdavis.edu/undergraduate/frequently-asked-questions'



print(list_of_scraped_links)


{'Advising for New Students': 'https://ece.ucdavis.edu/undergraduate/aggie-advising', 'IDP Program': 'https://ece.ucdavis.edu/undergraduate/integrated-degree-programs', 'College of Engineering': 'https://engineering.ucdavis.edu/undergraduates/academic-advising', 'Degree Requirements': 'https://ece.ucdavis.edu/undergraduate/degrees-requirements', 'Course Offerings': 'https://ece.ucdavis.edu/course-schedules', 'Mandatory Academic Advising': 'https://ece.sf.ucdavis.edu/undergraduate/annual-mandatory-advising', 'Release Holds': 'https://ece.sf.ucdavis.edu/undergraduate/annual-mandatory-advising', 'Internship Info.Forms': 'https://ece.sf.ucdavis.edu/undergraduate/forms', 'PTA Request': 'https://ece.ucdavis.edu/pta-request-1', "FAQ's": 'https://ece.ucdavis.edu/undergraduate/frequently-asked-questions', 'Change of Major': 'https://ece.ucdavis.edu/undergraduate/frequently-asked-questions'}


In [4]:
def check_info(item):
    #Email: "mailto:eceugradinfo@ucdavis.edu">eceugradinfo@ucdavis.edu</a><span>   </span>

    if 'mailto' in item:
        cleaned = item.replace('Email:', '').replace('\"mailto:', '').replace('\">eceugradinfo@ucdavis', '').replace('</a><span>', '').strip()
        cleaned = str(cleaned.split('.edu')[0]) + '.edu'
        return cleaned
    elif 'Click' in item:
        cleaned = item.replace('Click "', '')

        return cleaned.split('.edu')[0] + '.edu'
    elif 'Office' in item:
        # print(item.replace('Office Location: ', ''))
        return item.replace('Office Location: ', '').strip()
    
    elif " " == item:
        pass
    else:
        return item

In [5]:
#This is a database for the advisors and their info
#name, office location, email, Advises, Office hours (in person), office hours(remote), link to schedule appointment
councelor_to_info = {}
councelors_page = soup.find('div', class_ = 'block block-layout-builder block-inline-blocksf-basic panel')
councelor_buffer = councelors_page.find_all('div')
councelor_more_buffer = councelor_buffer[0]
list_ = councelor_more_buffer.find_all('p')
list2 = councelor_more_buffer.find_all('li')

list_of_needed_componenets = []
list2 = list2[2:]
list2 = list2[:-6]
cleaned_up = []

for li in list2:
    cleaned_up.append(str(li).replace('<li>', '').replace('</li>', '').replace('<span>', '').replace('</span>', ''))

# print(*cleaned_up, sep='\n')
    

for l in list_:
    text = str(l).replace('<p>', '').replace('</p>', '').replace('<a href=', '')+ '\n'
    list_of_needed_componenets.append(text)

new_list = list_of_needed_componenets[5:]
del new_list[5:7]
del new_list[5]
del new_list[10:14]



# print(len(new_list))

info = []
#councelor_to_info
i = 0
k=0
name = ""
for stuff in new_list:
    if i == 0 or i == 5 or i == 10:
        name = new_list[i].strip()
        councelor_to_info[name] = []
    else:
        item = check_info(stuff)
        if 'Advising' in item:
            # print('here')
            for j in range(3):
                if k == 5:
                    councelor_to_info[name].append(str(cleaned_up[k]).replace('\xa0', ' ').strip())
                    break
                else:
                    councelor_to_info[name].append(str(cleaned_up[k]).replace('\xa0', ' ').strip())
                k = k + 1
        else:
            councelor_to_info[name].append(item)
    
    i = i + 1


print(councelor_to_info)















{'Ileana Oseguera': ['2064 Kemper', 'eceugradinfo@ucdavis.edu', 'Students with last names: A-L', 'In-Person: Tuesday-Thursday: 9:00 am - 4:00 pm (by appointment)', 'Remote: Monday, Friday: 9:00 am - 4:00 pm (by appointment)', 'https://appointments.ucdavis.edu'], 'Pascal Gonzalez': ['2064 Kemper', 'eceugradinfo@ucdavis.edu', 'Students with last names: M-Z', 'In-Person: Mondays: 9:00 am - 4:00 pm (by appointment)', 'Remote: Tuesday-Wednesday: 9:00 am - 4:00 pm (by appointment)', 'https://appointments.ucdavis.edu'], 'Hussain Al-Asaad': ['eceugradinfo@ucdavis.edu', 'Website:\xa0"https://ece.ucdavis.edu/directory/hussain-al-asaad">https://www.ece.ucdavis.edu/~halasaad/</a>\n', 'Office hours:', '\xa0\n', '<strong>Engineering Undergraduate Office</strong>', 'Drop-in hours:\xa0"https://engineering.ucdavis.edu/undergraduates/academic-advising"><strong>see website</strong></a><br/>Location: Kemper 1050 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0

In [6]:
print(*list_of_scraped_links, sep='\n')

Advising for New Students
IDP Program
College of Engineering
Degree Requirements
Course Offerings
Mandatory Academic Advising
Release Holds
Internship Info.Forms
PTA Request
FAQ's
Change of Major


In [7]:
#This is for Degree Requirements

url = list_of_scraped_links['Degree Requirements']
base_url = "https://ece.ucdavis.edu"

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

main_section = soup.find('div', class_ = 'clearfix text-formatted field field--name-body field--type-text-with-summary field--label-hidden field__item')

important_links_in_main_section = main_section.find_all(href=True)

descrp_and_link = {}

for link in important_links_in_main_section:
    cleaned_up = str(link).replace('<a href=\"', '').replace("\"", '').replace('</a>', '')
    splitted = cleaned_up.split('>')
    descrp_and_link[splitted[1].replace('\xa0', ' ')] = base_url + splitted[0]

# print(descrp_and_link)

#---------------------------------------------------Classes Link----------------------------------------------------#

url = descrp_and_link['classes']

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

classs_sections_to_classes_offered_and_their_descriptions = {} #<-- [Lower Division Classes] = [Class 1, class 2, class 3] -> [units, learning stuff. blah, blah]
buff = soup.find_all('h3')
sections = []
for b in buff:
    cleaned = str(b).replace('<h3><a class="ck-anchor" id=\"', "")
    split = cleaned.split('>')
    sections.append(split[2].replace('</h3', '').strip())

all_groups = soup.find_all('ul', class_ = 'list--accordion')

group1 = all_groups[0]

class_names = []

groups_and_class_count = {}

count = 0
kindex = 0
for group in all_groups:
    for c in group.find_all('li'):
        if '<li><p>' not in str(c) and '<li><strong>' not in str(c) and '<span><strong>' not in str(c) and 'Units' not in str(c):
            if 'span' in str(c):
                splitted = str(c).split('<span>')
                extra_split = []
                for s in splitted:
                    if '<li>' not in s and 'id=' not in s:
                        extra_split.append(s.replace('</span>', '').replace('</li>', '').replace('\xa0', ''))
                index = 0
                for e in extra_split:
                    if e.endswith("–"):
                        extra_split[index] = extra_split[index] + extra_split[index + 1]
                        del extra_split[index + 1]
                        index = index - 1
                    elif 'Modern Parallel Computing' in e:
                        extra_split[index] = 'EEC 279' + extra_split[index]
                        index = index + 1
                    else:
                        index = index + 1
                # print(extra_split[0])
                class_names.append(extra_split[0])
                count = count + 1
                        

            else:
                cleaned_name = str(c).replace('<li>', '').replace('</li>', '').replace('<a class="ck-anchor" id="eng6"></a>','').replace('<a class="ck-anchor" id="eng17"></a>','').replace('<a class="ck-anchor" id="course-eec-201"></a>', '').replace('<a class="ck-anchor" id="course-eec-298"></a>', '')
                class_names.append(cleaned_name)
                # print(cleaned_name)
                count = count + 1


                
    groups_and_class_count[sections[kindex]] = count
    kindex = kindex + 1
    count = 0




print(groups_and_class_count)
print(len(sections))
print(len(class_names))



{'Lower-Division Undergraduate Courses': 11, 'Upper-Division Undergraduate Courses': 51, 'Graduate Courses': 84, 'Courses On Hold as of 2022': 5}
4
151


In [8]:
all_groups = soup.find_all('ul', class_ = 'list--accordion')

group1 = all_groups[0]

buf = group1.find_all('li')
# print(*buf, sep='\n')


count = 0
for b in buf:
    if '<li><p>' in str(b) or '<li><strong>' in str(b):
        count = count + 1
        print(b)

print(count)


<li><p><strong>Units:</strong> 4.0<br/><strong>Learning Activities: </strong><br/>     • Lecture - 3.0 <br/>     • Laboratory - 3.0 <br/><strong>Prerequisites:</strong> (MAT 16A C- or better or MAT 17A C- or better or MAT 19A C- or better or MAT 21A C- or better); (MAT 16B C- or better {can be concurrent} or MAT 17B C- or better {can be concurrent} or MAT 19B C- or better {can be concurrent} or MAT 21B C- or better {can be concurrent})<br/><strong>Catalog Description:</strong> Methodology for solving engineering problems. Engineering computing and visualization based on MATLAB. Engineering examples and applications. <em><strong>GE Credit:</strong></em> SE</p><p><span><strong>ABET Student Outcomes</strong></span><strong>:</strong><br/>1)    The student will be able to identify, formulate and solve engineering problems using modern computational methods. <br/>2)    Students who have completed this course should have achieved:<br/>       a)    Student Outcome 1: an ability to identify, fo